In [1]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
import math
import pmdarima as pm

from utility_code.utility import utils

#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")
dummydata = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')

train['Population'] = train['Confirmed'].divide(train['Incident_Rate'])*1e5
train['Active/Population'] = train['Active'].divide(train['Population'])
train['Deaths/Population'] = train['Deaths'].divide(train['Population'])
train['Active/People_Tested*Population'] = train['Active']/train['People_Tested']*train['Population']
train['Recovered/Population'] = train['Recovered']/train['Population']


confirmed = train['Confirmed'].to_list()
daily = [confirmed[i]-confirmed[i-50] for i in range(len(confirmed)) if i-50 >= 0]
for i in range(50):
    daily.insert(0,0)

train['New_Cases'] = daily

tests = train['People_Tested'].to_list()
dailyTests = [tests[i]-tests[i-50] for i in range(len(tests)) if i-50 >= 0]
for i in range(50):
    dailyTests.insert(0,0)

train['New_Tests'] = dailyTests

train['New_Cases/New_Tests'] = train['New_Cases']/train['New_Tests']

statesdata = {}
states = pd.Series.unique(train['Province_State'])
num_states = len(states)
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s ,:]

In [1]:
import pandas as pd
confirmed = statesdata['Wisconsin']['Confirmed']
deaths = statesdata['Wisconsin']['Deaths']
deaths = deaths.values
confirmed = confirmed.values
def difference(dataset, interval = 1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i]-dataset[i-interval]
        diff.append(value)
    return np.array(diff)

deaths = difference(deaths,1)

pd.plotting.autocorrelation_plot(deaths)

NameError: name 'statesdata' is not defined

In [6]:
import warnings
warnings.filterwarnings('ignore')
#p,d,q = 
days = 26
mapeTotC = 0
mapeTotD = 0

from statsmodels.tsa.arima_model import ARIMA
#from statsmodels.tsa.arima_model import SARIMAX

def predictARIMA(X, p, d, q, days):
    model = ARIMA(X, order=(p,d,q))
    model_fit = model.fit()
    start = len(X)
    forecast = model_fit.predict(start = start, end = start + days-1)
    return(forecast)

def predictARIMA2(X, p, d, q, days):
    arima = pm.auto_arima(X, error_action='ignore', trace=True, max_p=9, max_d=3, max_q=7,
                                  suppress_warnings=True, maxiter=25,
                                  seasonal= False, out_of_sample_size=10)
    forecast = arima.predict(n_periods = days)
    return(forecast)



def difference(dataset, interval = 1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i]-dataset[i-interval]
        diff.append(value)
    return np.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    #yhat = np.exp(yhat)
    return yhat + history[-1]

from statsmodels.tsa.arima.model import ARIMA
days = 30
proj = {}
p,d,q = 4,2,1
# for p in [2,4,6,8]:
#     for d in [2,3]:
#         for q in [1,3,5]:
            #if q  > p: continue
mapeTotC = 0
mapeTotD = 0 
for s in states:

    a = statesdata[s]
    a = a.reset_index()
    confirmed = a['Confirmed']
    deaths = a['Deaths']

    splitpt = len(confirmed) - days
    confirmed_train, confirmed_test = confirmed[0:splitpt], confirmed[splitpt:]
    deaths_train, deaths_test = deaths[0:splitpt], deaths[splitpt:]

    X = confirmed_train.values
    try:
        forecastC = predictARIMA2(X, p,d,q, days)
    except:
        print('Params:', p,d,q, 'Forecast C failed!')
        continue

    Y= deaths_train.values
    try:
        forecastD = predictARIMA2(Y, p,d,q, days)
    except:
        print('Params:', p,d,q, 'Forecast D failed!')
        continue

    df = {'Confirmed': forecastC, 'Deaths': forecastD}
    proj[s] = pd.DataFrame(df)

    mapeC = MAPE(forecastC, confirmed_test)

    mapeD = MAPE(forecastD, deaths_test)
    mapeTotC += mapeC
    mapeTotD += mapeD
    #print('For', s, '\n Confirmed MAPE:',mapeC, '\n Deaths MAPE:', mapeD)

print('Params:', p,d,q, 'MAPE C', mapeTotC, 'MAPE D', mapeTotD)
print('Projected MAPE: ', (mapeTotC + mapeTotD)/2)

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=1554.494, Time=0.14 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=1596.027, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=1569.844, Time=0.05 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=1549.813, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=1594.197, Time=0.02 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=1551.094, Time=0.11 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=1550.525, Time=0.11 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=1550.472, Time=0.14 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=1551.757, Time=0.04 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0] intercept
Total fit time: 0.716 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=890.979, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=880.442, Time=0.06 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=845.411, T

 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=1966.300, Time=0.19 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=1988.251, Time=0.17 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=1966.201, Time=0.17 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=1990.024, Time=0.08 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=1964.992, Time=0.07 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=1975.378, Time=0.05 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=1981.352, Time=0.06 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=1966.468, Time=0.11 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=1968.469, Time=0.10 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=1967.345, Time=0.07 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=1986.397, Time=0.05 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=1975.385, Time=0.16 sec

Best model:  ARIMA(1,2,1)(0,0,0)[0]          
Total fit time: 1.934 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=1069.447, Time=0.23 sec
 ARIMA(0,2,0)(0,0,0)[0

 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=858.615, Time=0.03 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=820.715, Time=0.19 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=815.518, Time=0.23 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=811.967, Time=0.28 sec
 ARIMA(5,2,0)(0,0,0)[0] intercept   : AIC=811.409, Time=0.31 sec
 ARIMA(6,2,0)(0,0,0)[0] intercept   : AIC=811.719, Time=0.28 sec
 ARIMA(5,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(4,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.35 sec
 ARIMA(6,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.47 sec
 ARIMA(5,2,0)(0,0,0)[0]             : AIC=809.428, Time=0.16 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=809.979, Time=0.15 sec
 ARIMA(6,2,0)(0,0,0)[0]             : AIC=809.748, Time=0.22 sec
 ARIMA(5,2,1)(0,0,0)[0]             : AIC=798.028, Time=0.27 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=796.014, Time=0.25 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC

 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=462.575, Time=0.12 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=504.205, Time=0.05 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=459.964, Time=0.27 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=464.382, Time=0.24 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=464.954, Time=0.17 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=463.245, Time=0.28 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=475.526, Time=0.22 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=466.261, Time=0.19 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=inf, Time=0.24 sec

Best model:  ARIMA(1,2,2)(0,0,0)[0] intercept
Total fit time: 2.538 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=1676.422, Time=0.30 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=1707.545, Time=0.05 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=1680.841, Time=0.10 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AI

 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=1365.293, Time=0.24 sec
 ARIMA(3,2,3)(0,0,0)[0] intercept   : AIC=1352.717, Time=0.30 sec
 ARIMA(2,2,4)(0,0,0)[0] intercept   : AIC=1355.081, Time=0.31 sec
 ARIMA(1,2,4)(0,0,0)[0] intercept   : AIC=1366.866, Time=0.26 sec
 ARIMA(3,2,4)(0,0,0)[0] intercept   : AIC=1369.543, Time=0.31 sec
 ARIMA(2,2,3)(0,0,0)[0]             : AIC=1351.305, Time=0.23 sec
 ARIMA(1,2,3)(0,0,0)[0]             : AIC=1364.069, Time=0.18 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=1358.498, Time=0.18 sec
 ARIMA(3,2,3)(0,0,0)[0]             : AIC=1351.917, Time=0.24 sec
 ARIMA(2,2,4)(0,0,0)[0]             : AIC=1354.687, Time=0.25 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=1383.386, Time=0.16 sec
 ARIMA(1,2,4)(0,0,0)[0]             : AIC=1366.133, Time=0.22 sec
 ARIMA(3,2,2)(0,0,0)[0]             : AIC=1360.745, Time=0.20 sec
 ARIMA(3,2,4)(0,0,0)[0]             : AIC=1368.600, Time=0.29 sec

Best model:  ARIMA(2,2,3)(0,0,0)[0]          
Total fit time: 4.954 seconds

...

 ARIMA(1,2,2)(0,0,0)[0]             : AIC=1594.437, Time=0.17 sec
 ARIMA(0,2,3)(0,0,0)[0]             : AIC=1600.391, Time=0.18 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=1587.768, Time=0.16 sec
 ARIMA(1,2,3)(0,0,0)[0]             : AIC=inf, Time=0.22 sec

Best model:  ARIMA(0,2,2)(0,0,0)[0]          
Total fit time: 2.718 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.27 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=1635.568, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=1604.227, Time=0.07 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=1570.770, Time=0.13 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=1633.675, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=1575.018, Time=0.17 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=1574.533, Time=0.19 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.20 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=1578.090, Time=0.07 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0] 

 ARIMA(0,2,0)(0,0,0)[0]             : AIC=640.295, Time=0.03 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=599.644, Time=0.21 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=595.655, Time=0.13 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=582.797, Time=0.24 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=578.867, Time=0.29 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=584.439, Time=0.24 sec
 ARIMA(3,2,3)(0,0,0)[0] intercept   : AIC=581.044, Time=0.35 sec
 ARIMA(2,2,4)(0,0,0)[0] intercept   : AIC=585.210, Time=0.39 sec
 ARIMA(1,2,4)(0,0,0)[0] intercept   : AIC=583.310, Time=0.35 sec
 ARIMA(3,2,4)(0,0,0)[0] intercept   : AIC=586.623, Time=0.44 sec
 ARIMA(2,2,3)(0,0,0)[0]             : AIC=578.254, Time=0.34 sec
 ARIMA(1,2,3)(0,0,0)[0]             : AIC=582.497, Time=0.22 sec
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=580.256, Time=0.26 sec
 ARIMA(3,2,3)(0,0,0)[0]             : AIC=579.410, Time=0.32 sec
 ARIMA(2,2,4)(0,0,0)[0]             : AIC=583.325, Time=0.31 sec
 ARIMA(1,2,2)(0,0,0)[0]  

 ARIMA(1,2,3)(0,0,0)[0]             : AIC=1308.536, Time=0.21 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=1309.973, Time=0.18 sec
 ARIMA(3,2,3)(0,0,0)[0]             : AIC=1309.153, Time=0.26 sec

Best model:  ARIMA(2,2,2)(0,0,0)[0]          
Total fit time: 4.178 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=728.298, Time=0.23 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=781.860, Time=0.03 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=765.774, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=725.247, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=779.860, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=727.003, Time=0.10 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=726.885, Time=0.13 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=727.573, Time=0.15 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=725.561, Time=0.04 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0] intercept
Total fit time: 0.876 seconds
Performing stepwise 

 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=508.967, Time=0.19 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=586.284, Time=0.04 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=559.227, Time=0.07 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=584.379, Time=0.03 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=508.669, Time=0.23 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=507.672, Time=0.14 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=507.135, Time=0.17 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=509.198, Time=0.23 sec
 ARIMA(0,2,4)(0,0,0)[0] intercept   : AIC=509.329, Time=0.22 sec
 ARIMA(1,2,4)(0,0,0)[0] intercept   : AIC=511.733, Time=0.31 sec
 ARIMA(0,2,3)(0,0,0)[0]             : AIC=507.740, Time=0.08 sec

Best model:  ARIMA(0,2,3)(0,0,0)[0] intercept
Total fit time: 1.849 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=705.252, Time=0.23 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : A

 ARIMA(0,2,1)(0,0,0)[0]             : AIC=778.792, Time=0.05 sec
 ARIMA(0,2,2)(0,0,0)[0]             : AIC=780.272, Time=0.07 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=811.830, Time=0.05 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=782.139, Time=0.12 sec

Best model:  ARIMA(0,2,1)(0,0,0)[0]          
Total fit time: 4.680 seconds
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.22 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=1159.640, Time=0.04 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=1123.294, Time=0.08 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=1096.659, Time=0.07 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=1157.725, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=1102.223, Time=0.13 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=1101.251, Time=0.14 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=1104.136, Time=0.14 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=1096.744, Time=0.05 sec

Best model:  ARIMA(0,2,1)(0,0,

Params: 1 1 1 MAPE C 1.9461118381882339 MAPE D 2.3628135502297036

Projected MAPE:  2.1544626942089686

Params: 1 2 1 MAPE C 2.2519528609901176 MAPE D 2.216307406131526

Projected MAPE:  2.234130133560822

Params: 1 3 1 MAPE C 3.1003665628066246 MAPE D 3.9856347983119753

Projected MAPE:  3.5430006805592997

Params: 3 1 1 MAPE C 2.021920504259834 MAPE D 2.6264695954414408

Projected MAPE:  2.3241950498506374

Params: 3 1 3 MAPE C 1.9839390738495128 MAPE D 2.3248175410550513

Projected MAPE:  2.154378307452282

Params: 3 2 1 MAPE C 2.2579889332648904 MAPE D 2.2834652120621675

Projected MAPE:  2.270727072663529

Params: 3 2 3 MAPE C 2.327875374338098 MAPE D 2.348015130749768

Projected MAPE:  2.337945252543933

Params: 3 3 1 MAPE C 2.7761672207063235 MAPE D 3.6465322758645957

Projected MAPE:  3.2113497482854596

Params: 3 3 3 MAPE C 3.0820265024511144 MAPE D 3.483869249703558

Projected MAPE:  3.282947876077336

Params: 5 1 1 MAPE C 1.984169515110471 MAPE D 2.3591148015219483

Projected MAPE:  2.1716421583162098

In [8]:
deaths_test

112    1627
113    1633
114    1666
115    1695
116    1714
117    1735
118    1755
119    1768
120    1797
121    1847
122    1882
123    1890
124    1893
125    1896
126    1898
127    1925
128    1936
129    1944
130    1974
131    1996
132    2011
133    2013
134    2024
135    2037
136    2045
137    2076
138    2107
139    2152
140    2162
141    2182
Name: Deaths, dtype: int64

In [5]:
# def MAPE(pred, valid):
#     pred = pred.reset_index()
#     valid = valid.reset_index()
#     pred = pred.astype('int64')
#     valid = valid.astype('int64')
#     v = pred.subtract(valid)
#     v = v.divide(valid)
#     v = v.abs()
#     v = v.sum(axis = 0)
#     #print(v)
#     #v = v[1]+v[2]
#     n = len(pred)
#     #print(len(pred))
#     return v/(2*n)
# #
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))
#MAPE(submission[['Confirmed']], validation[['Confirmed']])

In [11]:
forecastD

array([1626.1662193 , 1648.30194226, 1670.21701713, 1692.22644035,
       1714.28334957, 1736.33687816, 1758.38344843, 1780.42912687,
       1802.47555763, 1824.52225181, 1846.56889683, 1868.61549739,
       1890.66209591, 1912.70869994, 1934.75530531, 1956.8019102 ,
       1978.84851482, 2000.89511946, 2022.94172413, 2044.98832881,
       2067.03493348, 2089.08153816, 2111.12814283, 2133.1747475 ,
       2155.22135217, 2177.26795685, 2199.31456152, 2221.36116619,
       2243.40777087, 2265.45437554])

Without the trend parameter for confirmed: 0.22241363123139815
Without the trend parameter for deaths: 0.26647074317976094
    
With the params, 4,2,3 for confirmed: 0.21210243608979962
With the 4,2,3 for deaths: 0.26341990030777807
    
With the params, 2,2,3 for confirmed: 0.1893949194911668
With the 2,2,3 for deaths: 0.25949362504775464
    
With the 6,2,3 for deaths: 0.24447115807485809
    
With the params, 4,2,1 for confirmed: 0.20066623728045355
With the 4,2,1 for deaths: 0.23821759832698347
    
With the params 4,2,0 for confirmed: 0.14797518105639543
With the parms above for deaths: 0.2358201596243279
    
With the params 1,1,1 for confirmed: 0.17404089441983467
With the parms above for deaths: 0.26697120703684923
    
With the params 2,2,2 for confirmed: 0.18581905974534663
With the parms above for deaths: 0.2611015188110797

With the params 4,2,2 for confirmed: 0.14797518105639543
With the parms above but also log for deaths: 0.22118460969592046

In [17]:
trainrd2 = pd.read_csv("train_round2.csv")
datestofind = trainrd2['Date'] == '09-01-2020'
start = datestofind[datestofind == True].index[0]
datestofind2 = trainrd2['Date'] == '09-10-2020'
end = datestofind[datestofind == True].index[49]

validation = trainrd2.iloc[start:end,:]
validation

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
7100,7100,Alabama,09-01-2020,127616,2200,48028.0,77388.0,2602.716398,958920.0,NaN,1.723922,19557.083814,NaN
7101,7101,Alaska,09-01-2020,5297,39,2246.0,3012.0,724.083959,374255.0,NaN,0.736266,51159.532223,NaN
7102,7102,Arizona,09-01-2020,202342,5044,30841.0,166457.0,2779.912998,1208990.0,NaN,2.492809,16609.932767,NaN
7103,7103,Arkansas,09-01-2020,61497,814,55647.0,5036.0,2037.806299,731025.0,NaN,1.323642,24223.740177,NaN
7104,7104,California,09-01-2020,715617,13150,NaN,702467.0,1811.128167,11470696.0,NaN,1.837575,29030.753344,NaN
7105,7105,Colorado,09-01-2020,57755,1946,5953.0,49856.0,1002.911055,1009970.0,NaN,3.369405,17538.050017,NaN
7106,7106,Connecticut,09-01-2020,53006,4466,8967.0,39573.0,1486.724631,1192338.0,NaN,8.425461,33442.973876,NaN
7107,7107,Delaware,09-01-2020,17535,605,9419.0,7511.0,1800.744328,238423.0,NaN,3.450242,24484.680066,NaN
7108,7108,Florida,09-01-2020,631040,11374,NaN,619666.0,2938.112148,4675866.0,NaN,1.802421,21770.757320,NaN
7109,7109,Georgia,09-01-2020,272697,5733,NaN,266964.0,2568.391596,2368498.0,NaN,2.102333,22307.654127,NaN


In [67]:
print('MAPE C', mapeTotC, 'MAPE D', mapeTotD)

MAPE C 0.370906263226227 MAPE D 0.41418819912041843


In [ ]:
testC